In [1]:
import matplotlib.pyplot as plt

from dataset.dataset import get_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from tqdm import tqdm
import pathlib
from final.matcher import OIE_Match

In [2]:
nlp = spacy.load("pt_core_news_lg")
matcher = OIE_Match()

In [3]:
def save_data(dataset, name: str):
    dir = 'datasets/outputs/analisado'
    path = pathlib.Path('datasets/outputs/analisado')
    path.mkdir(parents=True, exist_ok=True)
    file_text = ""
    before_tag = '\tXX\t-\t-\t-\t-\t-\t*\t'
    for key in tqdm(dataset, desc='saving data'):
        ext = dataset[key]['ext']['splited']
        sent = dataset[key]['sent']['sent']
        match = matcher.match(sent=sent, 
                              arg1=ext['arg0'], 
                              rel=ext['rel'],
                              arg2=ext['arg1'])
        if match[-1]:
            for i,token in enumerate([token.text for token in nlp(sent)]):
                if i in range(match[0][0], match[0][1]+1):
                    if len(range(match[0][0], match[0][1]+1)) == 1:
                        file_text += token + before_tag + "S-ARG0" + "\n"
                    elif i == match[0][0]:
                        file_text += token + before_tag + "B-ARG0" + "\n"
                    elif i > match[0][0] and i<match[0][1]:
                        file_text += token + before_tag + "I-ARG0" + "\n"
                    elif i == match[0][1]:
                        file_text += token + before_tag + "E-ARG0" + "\n"

                elif i in range(match[1][0], match[1][1]+1):
                    if len(range(match[1][0], match[1][1] + 1)) == 1:
                        file_text += token + before_tag + "S-V" + "\n"
                    elif i == match[1][0]:
                        file_text += token + before_tag + "B-V" + "\n"
                    elif i > match[1][0] and i < match[1][1]:
                        file_text += token + before_tag + "I-V" + "\n"
                    elif i == match[1][1]:
                        file_text += token + before_tag + "E-V" + "\n"

                elif i in range(match[2][0], match[2][1]+1):
                    if len(range(match[2][0], match[2][1] + 1)) == 1:
                        file_text += token + before_tag + "S-ARG1" + "\n"
                    elif i == match[2][0]:
                        file_text += token + before_tag + "B-ARG1" + "\n"
                    elif i > match[2][0] and i < match[2][1]:
                        file_text += token + before_tag + "I-ARG1" + "\n"
                    elif i == match[2][1]:
                        file_text += token + before_tag + "E-ARG1" + "\n"
                else:
                    file_text += token + before_tag + "O" + "\n"
            file_text += "\n"
    with open(dir + "/" + name + ".txt", "a", encoding="utf-8") as f:
        try:
            txt_f = f.read()
        except:
            txt_f = ""
        txt_f += file_text
        f.write(txt_f)

In [4]:
def load_dataset(): 
    dataset = get_dataset()
    extractions = {}
    i = 0
    split = 0
    while split<=1:
        for triple in dataset[split]:
            sent = triple.phrase
            sent_counter = 0
            for token in nlp(sent):
                sent_counter += 1
                
            ext = triple.gold_extractions[0]
            merged = ext.arg0 + " " + ext.rel + " " + ext.arg1
            ext_counter = 0
            for token in nlp(merged):
                ext_counter += 1
                
            ext = {"length" : ext_counter,
                   "merged": merged,
                   "splited" : {
                       "arg0" : ext.arg0,
                       "rel" : ext.rel,
                       "arg1" : ext.arg1
                        }
                   }
            extractions[i] = {"sent" : {"sent" : sent,
                                        "length" : sent_counter
                                        },
                              "ext" : ext}
            i+=1
        split+=1
    return extractions

In [4]:
dataset = load_dataset()
dataset[0]

processando TA_dev: 100%|██████████| 927/927 [00:05<00:00, 177.09it/s]


In [5]:
print(dataset)

NameError: name 'dataset' is not defined

In [7]:
same_string_extractions = {}
minimum_extractions = {}
small_extractions = {}
long_extractions = {}
small_sentences = {}

long_sentences = {}
long_sentece_small_extractions = {}

for key in dataset:
    inst = dataset[key]
    sent_len = inst["sent"]["length"]
    ext_len = inst["ext"]["length"]
    if sent_len-1 == ext_len or sent_len == ext_len:
        same_string_extractions[key] = inst
    elif ext_len > 8:
        long_extractions[key] = inst
    elif ext_len == 3:
        minimum_extractions[key] = inst
    elif ext_len > 3 and ext_len <=10:
        small_extractions[key] = inst
    

In [9]:
print("extractions with same sentence length: ",len(same_string_extractions))
print("extractions with one tag on arg0, one on rel and one on arg1: ", len(minimum_extractions))
print("small extractions, token counter <= 8: ", len(small_extractions))
print("long extractions, token counter > 8: ", len(long_extractions))

extractions with same sentence length:  16463
extractions with one tag on arg0, one on rel and one on arg1:  927
small extractions, token counter <= 8:  38553
long extractions, token counter > 8:  61959


In [10]:
save_data(small_extractions, 'small')

saving data: 100%|██████████| 38553/38553 [08:56<00:00, 71.83it/s]
